# Addition_Subtraction_rnn

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


## Data Representation

In [2]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
            
        #print(x)
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [3]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [4]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+- '
ctable = CharacterTable(chars)
print(ctable)

## Data Generation

In [5]:
def Generate_Data(DIGITS, DATA_SIZE, MAXLEN):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE / 2:    ## the amount of generated data is TRAINING_SIZE
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()

        if(a+b>=c):   #####
            # Skip any addition questions we've already seen
            # Also skip any such that x+Y == Y+x (hence the sorting).
            key = tuple(sorted((a, b, c)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}+{}-{}'.format(a, b, c)    #####
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a + b - c)   #####
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)

    while len(questions) >= DATA_SIZE / 2 and len(questions) < DATA_SIZE:    ## the amount of generated data is TRAINING_SIZE
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()

        if(a>b):   #####
            # Skip any addition questions we've already seen
            # Also skip any such that x+Y == Y+x (hence the sorting).
            key = tuple(sorted((a, b, c)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}-{}+{}'.format(a, b, c)    #####
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a - b + c)   #####
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

## Feature Engineering

In [6]:
def Vectorization(questions, expected):
    print('Vectorization...')
    x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
    y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, MAXLEN)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, DIGITS + 1)

    return x, y

## Get Training Data and Validation Data
* total data = 50000, 45000 for training, 5000 for validation

In [7]:
questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 11, 13)
(45000, 4, 13)
Validation Data:
(5000, 11, 13)
(5000, 4, 13)


## Build Model

In [8]:
def Build_Model(RNN, HIDDEN_SIZE, BATCH_SIZE, LAYERS, MAXLEN, DIGITS, chars):
    print('Build model...')
    model = Sequential()
    
    model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    model.add(layers.Dense((DIGITS + 1) * 512))
    model.add(layers.Reshape(((DIGITS + 1), 512)))
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.summary()
    
    return model

In [9]:
def Train_Step(model, epoch, REVERSE = True):
    for iteration in range(1, epoch):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        model.fit(x_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=1,
                  validation_data=(x_val, y_val))
        # Select 10 samples from the validation set at random so we can visualize
        # errors.
        for i in range(10):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            preds = model.predict_classes(rowx, verbose=0)
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])
            guess = ctable.decode(preds[0], calc_argmax=False)
            print('Q', q[::-1] if REVERSE else q, end=' ')
            print('T', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print(guess)

## String Matching
* training 200 epochs

In [14]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 512)               1077248   
_________________________________________________________________
dense_5 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
reshape_3 (Reshape)          (None, 4, 512)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 4, 512)            2099200   
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4, 13)             6669      
Total params: 4,233,741
Trainable params: 4,233,741
Non-trainable params: 0
_________________________________________________________________


In [15]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 324us/step - loss: 1.7622 - acc: 0.3775 - val_loss: 1.6442 - val_acc: 0.4046
Q 363-74+73   T 362  ☒ 328 
Q 40+413-44   T 409  ☒ 304 
Q 4+226-56    T 174  ☒ 30  
Q 905+7-46    T 866  ☒ 102 
Q 280+7-86    T 201  ☒ 20  
Q 803-6+9     T 806  ☒ 803 
Q 332+6-7     T 331  ☒ 301 
Q 77+812-6    T 883  ☒ 722 
Q 54+63-7     T 110  ☒ 55  
Q 667-646+3   T 24   ☒ 30  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 1.4672 - acc: 0.4556 - val_loss: 1.2713 - val_acc: 0.5147
Q 3-0+39      T 42   ☑ 42  
Q 502-79+347  T 770  ☒ 805 
Q 16-6+854    T 864  ☒ 886 
Q 975+3-18    T 960  ☒ 936 
Q 671+538-442 T 767  ☒ 100 
Q 902-5+35    T 932  ☒ 960 
Q 659+0-8     T 651  ☒ 658 
Q 473+64-312  T 225  ☒

45000/45000 [==============================] - 12s 272us/step - loss: 0.4070 - acc: 0.8481 - val_loss: 0.4268 - val_acc: 0.8379
Q 441-21+5    T 425  ☒ 426 
Q 939-610+3   T 332  ☑ 332 
Q 814-1+963   T 1776 ☒ 1777
Q 775-90+918  T 1603 ☒ 1695
Q 98-23+1     T 76   ☒ 77  
Q 937-515+80  T 502  ☒ 492 
Q 159+455-2   T 612  ☒ 510 
Q 101-58+651  T 694  ☒ 799 
Q 99-8+283    T 374  ☑ 374 
Q 28+7-27     T 8    ☒ 9   

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step - loss: 0.3663 - acc: 0.8625 - val_loss: 0.3503 - val_acc: 0.8682
Q 58-0+778    T 836  ☑ 836 
Q 812-679+2   T 135  ☑ 135 
Q 67-36+515   T 546  ☑ 546 
Q 0+711-7     T 704  ☑ 704 
Q 90-5+495    T 580  ☒ 570 
Q 393-89+0    T 304  ☒ 296 
Q 796+56-0    T 852  ☒ 842 
Q 63-43+5     T 25   ☑ 25  
Q 218-7+3     T 214  ☑ 214 
Q 51+959-17   T 993  ☑ 993 

--------------------------------------------------
Iteratio

Q 39-35+6     T 10   ☑ 10  
Q 692-76+372  T 988  ☑ 988 
Q 523+4-31    T 496  ☑ 496 
Q 231+37-90   T 178  ☑ 178 
Q 67-3+50     T 114  ☑ 114 
Q 79-62+0     T 17   ☑ 17  
Q 818+1-72    T 747  ☑ 747 
Q 77-25+3     T 55   ☑ 55  
Q 449-19+7    T 437  ☑ 437 
Q 86-1+98     T 183  ☑ 183 

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 0.1300 - acc: 0.9549 - val_loss: 0.1760 - val_acc: 0.9340
Q 578+57-10   T 625  ☒ 615 
Q 921-231+217 T 907  ☒ 917 
Q 150+243-5   T 388  ☑ 388 
Q 170+31-0    T 201  ☑ 201 
Q 29+780-399  T 410  ☑ 410 
Q 79-50+3     T 32   ☑ 32  
Q 544-151+275 T 668  ☑ 668 
Q 78-51+37    T 64   ☒ 74  
Q 9+64-6      T 67   ☑ 67  
Q 655+781-0   T 1436 ☑ 1436

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 272us/step - 

45000/45000 [==============================] - 12s 272us/step - loss: 0.0712 - acc: 0.9765 - val_loss: 0.1320 - val_acc: 0.9553
Q 998+262-6   T 1254 ☑ 1254
Q 801-664+24  T 161  ☒ 171 
Q 6+50-40     T 16   ☑ 16  
Q 996+80-2    T 1074 ☑ 1074
Q 33+4-2      T 35   ☑ 35  
Q 982+33-5    T 1010 ☑ 1010
Q 6-4+72      T 74   ☑ 74  
Q 753-8+22    T 767  ☑ 767 
Q 9+862-21    T 850  ☑ 850 
Q 663-0+582   T 1245 ☑ 1245

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 270us/step - loss: 0.0897 - acc: 0.9688 - val_loss: 0.1655 - val_acc: 0.9463
Q 544+315-39  T 820  ☑ 820 
Q 772-44+352  T 1080 ☑ 1080
Q 97+578-86   T 589  ☒ 599 
Q 6-1+201     T 206  ☑ 206 
Q 23-9+668    T 682  ☑ 682 
Q 798-7+79    T 870  ☑ 870 
Q 685-60+15   T 640  ☑ 640 
Q 46+44-8     T 82   ☑ 82  
Q 76+47-16    T 107  ☑ 107 
Q 842+194-88  T 948  ☑ 948 

--------------------------------------------------
Iteratio

Q 51-2+367    T 416  ☑ 416 
Q 776-6+424   T 1194 ☑ 1194
Q 456-137+69  T 388  ☑ 388 
Q 0+388-208   T 180  ☒ 181 
Q 968-8+82    T 1042 ☑ 1042
Q 50+69-52    T 67   ☑ 67  
Q 689-82+342  T 949  ☑ 949 
Q 35+6-20     T 21   ☑ 21  
Q 308-66+552  T 794  ☑ 794 
Q 2+332-2     T 332  ☑ 332 

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step - loss: 0.0634 - acc: 0.9786 - val_loss: 0.1513 - val_acc: 0.9488
Q 2+732-53    T 681  ☑ 681 
Q 87-11+832   T 908  ☑ 908 
Q 211+84-279  T 16   ☒ 10  
Q 594-487+259 T 366  ☑ 366 
Q 627+2-82    T 547  ☑ 547 
Q 993+3-586   T 410  ☑ 410 
Q 8+6-6       T 8    ☑ 8   
Q 59+622-348  T 333  ☑ 333 
Q 714-9+239   T 944  ☑ 944 
Q 107-86+1    T 22   ☒ 21  

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step - 

...

45000/45000 [==============================] - 12s 271us/step - loss: 0.0220 - acc: 0.9931 - val_loss: 0.0957 - val_acc: 0.9732
Q 191+1-27    T 165  ☑ 165 
Q 660+37-73   T 624  ☑ 624 
Q 42+911-32   T 921  ☑ 921 
Q 663+9-82    T 590  ☑ 590 
Q 689+994-53  T 1630 ☑ 1630
Q 32+17-7     T 42   ☑ 42  
Q 86-68+5     T 23   ☑ 23  
Q 67+212-5    T 274  ☑ 274 
Q 57+52-84    T 25   ☑ 25  
Q 326-92+5    T 239  ☑ 239 

--------------------------------------------------
Iteration 136
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step - loss: 0.0105 - acc: 0.9975 - val_loss: 0.0860 - val_acc: 0.9758
Q 4-1+365     T 368  ☑ 368 
Q 6+890-62    T 834  ☑ 834 
Q 10-1+27     T 36   ☑ 36  
Q 1+570-0     T 571  ☑ 571 
Q 64+944-682  T 326  ☑ 326 
Q 287+404-161 T 530  ☒ 520 
Q 755-8+9     T 756  ☑ 756 
Q 534-6+73    T 601  ☑ 601 
Q 17-1+6      T 22   ☑ 22  
Q 65-1+197    T 261  ☑ 261 

--------------------------------------------------
Iterati

Q 91+407-272  T 226  ☑ 226 
Q 6+169-2     T 173  ☑ 173 
Q 241-55+41   T 227  ☑ 227 
Q 871+3-42    T 832  ☑ 832 
Q 2+49-7      T 44   ☑ 44  
Q 108-95+8    T 21   ☒ 30  
Q 766-4+785   T 1547 ☑ 1547
Q 766-9+77    T 834  ☑ 834 
Q 17-6+92     T 103  ☑ 103 
Q 8+415-84    T 339  ☑ 339 

--------------------------------------------------
Iteration 149
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 270us/step - loss: 0.0200 - acc: 0.9940 - val_loss: 0.1060 - val_acc: 0.9702
Q 679+37-258  T 458  ☑ 458 
Q 387+2-48    T 341  ☑ 341 
Q 25-7+0      T 18   ☑ 18  
Q 658-228+3   T 433  ☑ 433 
Q 21-5+628    T 644  ☑ 644 
Q 92-62+809   T 839  ☑ 839 
Q 225+5-6     T 224  ☑ 224 
Q 22-7+77     T 92   ☑ 92  
Q 560-410+332 T 482  ☑ 482 
Q 125-1+276   T 400  ☑ 400 

--------------------------------------------------
Iteration 150
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 270us/step 

45000/45000 [==============================] - 12s 274us/step - loss: 0.0165 - acc: 0.9951 - val_loss: 0.1171 - val_acc: 0.9712
Q 279+9-8     T 280  ☑ 280 
Q 216-5+30    T 241  ☑ 241 
Q 568+78-47   T 599  ☑ 599 
Q 0+114-3     T 111  ☑ 111 
Q 25-13+82    T 94   ☑ 94  
Q 502-19+62   T 545  ☑ 545 
Q 7-0+336     T 343  ☑ 343 
Q 90+440-13   T 517  ☑ 517 
Q 22+94-1     T 115  ☑ 115 
Q 649+58-325  T 382  ☑ 382 

--------------------------------------------------
Iteration 163
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 0.0407 - acc: 0.9869 - val_loss: 0.1104 - val_acc: 0.9682
Q 610+52-457  T 205  ☑ 205 
Q 4+242-8     T 238  ☑ 238 
Q 4+951-19    T 936  ☑ 936 
Q 170+31-0    T 201  ☑ 201 
Q 8-1+306     T 313  ☑ 313 
Q 219-0+8     T 227  ☑ 227 
Q 162-89+930  T 1003 ☒ 103 
Q 80+756-4    T 832  ☑ 832 
Q 692-76+372  T 988  ☑ 988 
Q 65+592-35   T 622  ☑ 622 

--------------------------------------------------
Iterati

45000/45000 [==============================] - 12s 271us/step - loss: 0.0315 - acc: 0.9901 - val_loss: 0.1105 - val_acc: 0.9688
Q 326-9+9     T 326  ☑ 326 
Q 30-1+14     T 43   ☑ 43  
Q 505-0+313   T 818  ☑ 818 
Q 70-11+413   T 472  ☑ 472 
Q 560+4-6     T 558  ☑ 558 
Q 212-59+0    T 153  ☑ 153 
Q 123-5+5     T 123  ☑ 123 
Q 97-7+5      T 95   ☑ 95  
Q 840-1+5     T 844  ☑ 844 
Q 56-1+257    T 312  ☑ 312 

--------------------------------------------------
Iteration 176
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 272us/step - loss: 0.0150 - acc: 0.9954 - val_loss: 0.0834 - val_acc: 0.9781
Q 715-1+67    T 781  ☑ 781 
Q 36-7+802    T 831  ☑ 831 
Q 77+812-6    T 883  ☑ 883 
Q 660-1+39    T 698  ☑ 698 
Q 594+21-9    T 606  ☑ 606 
Q 4+64-37     T 31   ☑ 31  
Q 22+774-31   T 765  ☑ 765 
Q 890-1+782   T 1671 ☑ 1671
Q 260-5+43    T 298  ☑ 298 
Q 831-349+36  T 518  ☑ 518 

--------------------------------------------------
Iterati

Q 87-61+459   T 485  ☑ 485 
Q 23+778-81   T 720  ☑ 720 
Q 204+585-164 T 625  ☑ 625 
Q 212-7+871   T 1076 ☑ 1076
Q 25+21-14    T 32   ☑ 32  
Q 20-9+334    T 345  ☑ 345 
Q 202-0+512   T 714  ☑ 714 
Q 290-9+90    T 371  ☒ 372 
Q 450-68+792  T 1174 ☑ 1174
Q 619-6+7     T 620  ☑ 620 

--------------------------------------------------
Iteration 189
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 270us/step - loss: 0.0236 - acc: 0.9924 - val_loss: 0.1085 - val_acc: 0.9722
Q 78+43-0     T 121  ☑ 121 
Q 76+6-7      T 75   ☑ 75  
Q 148+0-1     T 147  ☑ 147 
Q 393-89+0    T 304  ☑ 304 
Q 46+795-8    T 833  ☑ 833 
Q 328-93+567  T 802  ☑ 802 
Q 50+556-0    T 606  ☑ 606 
Q 2+317-3     T 316  ☑ 316 
Q 692-95+6    T 603  ☑ 603 
Q 180+18-25   T 173  ☑ 173 

--------------------------------------------------
Iteration 190
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 269us/step 

## Testing

In [16]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [17]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 908+3-3     T 908  ☑ 908 
Q 15+744-61   T 698  ☑ 698 
Q 970+2-7     T 965  ☑ 965 
Q 60+369-78   T 351  ☑ 351 
Q 6+3-6       T 3    ☑ 3   
Q 351+4-95    T 260  ☒ 250 
Q 5+8-7       T 6    ☑ 6   
Q 942+6-21    T 927  ☑ 927 
Q 5+3-3       T 5    ☑ 5   
Q 72+1-5      T 68   ☑ 68  
Testing Accuracy :  0.906


## Result
### * 利用LSTM做encoder-decoder的seq2seq架構來實現加減混合
### * Training data總數為50000筆，45000為training，5000為validation
### * Batch size = 128
### * RNN hidden layer = 512
### * Optimizer使用adam
### * Epochs = 300
### * Validation result可達97.82%
### * Testing data總數為1000筆，Testing accuracy可達 90.6%

## Other Discussion
* 實驗使用不同的epoch和batch size訓練
* 實驗不同位數的數字
* 實驗"more number subtract"

## 1.1 實驗使用不同的epoch和batch size訓練
* with batch = 64, epoch = 200

In [18]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 64
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 512)               1077248   
_________________________________________________________________
dense_7 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
reshape_4 (Reshape)          (None, 4, 512)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 512)            2099200   
_________________________________________________________________
time_distributed_4 (TimeDist (None, 4, 13)             6669      
Total params: 4,233,741
Trainable params: 4,233,741
Non-trainable params: 0
_________________________________________________________________


In [19]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 25s 549us/step - loss: 1.6830 - acc: 0.3985 - val_loss: 1.4183 - val_acc: 0.4718
Q 61+133-57   T 137  ☒ 22  
Q 9-2+537     T 544  ☒ 408 
Q 6+36-20     T 22   ☒ 2   
Q 2+66-16     T 52   ☒ 19  
Q 857-96+384  T 1145 ☒ 131 
Q 366+37-3    T 400  ☒ 366 
Q 532+38-7    T 563  ☒ 527 
Q 2-0+666     T 668  ☒ 69  
Q 966-545+362 T 783  ☒ 709 
Q 2+732-53    T 681  ☒ 60  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 495us/step - loss: 1.1932 - acc: 0.5464 - val_loss: 1.0389 - val_acc: 0.5975
Q 879-58+1    T 822  ☒ 830 
Q 410-34+246  T 622  ☒ 528 
Q 64-31+19    T 52   ☒ 50  
Q 652-43+90   T 699  ☒ 690 
Q 44-2+16     T 58   ☒ 50  
Q 1+666-242   T 425  ☒ 360 
Q 97+47-6     T 138  ☒ 130 
Q 950-802+182 T 330  ☒

45000/45000 [==============================] - 22s 489us/step - loss: 0.3713 - acc: 0.8609 - val_loss: 0.4709 - val_acc: 0.8146
Q 804-25+2    T 781  ☑ 781 
Q 79+9-27     T 61   ☒ 62  
Q 28+50-0     T 78   ☑ 78  
Q 91-5+5      T 91   ☑ 91  
Q 421+97-2    T 516  ☑ 516 
Q 862-8+627   T 1481 ☒ 1480
Q 576-1+268   T 843  ☒ 840 
Q 44+9-41     T 12   ☑ 12  
Q 275+474-3   T 746  ☑ 746 
Q 85+501-317  T 269  ☒ 260 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 491us/step - loss: 0.3471 - acc: 0.8714 - val_loss: 0.3785 - val_acc: 0.8542
Q 470-7+528   T 991  ☒ 999 
Q 186-6+375   T 555  ☒ 553 
Q 129-90+891  T 930  ☒ 922 
Q 740-60+86   T 766  ☒ 765 
Q 12+9-8      T 13   ☑ 13  
Q 960+2-85    T 877  ☒ 876 
Q 45-4+46     T 87   ☑ 87  
Q 208-97+2    T 113  ☑ 113 
Q 639-627+57  T 69   ☒ 76  
Q 82-68+5     T 19   ☒ 29  

--------------------------------------------------
Iteratio

Q 51-10+2     T 43   ☑ 43  
Q 853-8+4     T 849  ☑ 849 
Q 6-3+815     T 818  ☑ 818 
Q 981+44-38   T 987  ☑ 987 
Q 5+496-93    T 408  ☑ 408 
Q 8+16-0      T 24   ☑ 24  
Q 257-96+535  T 696  ☑ 696 
Q 48+604-2    T 650  ☒ 659 
Q 385+86-98   T 373  ☑ 373 
Q 6+87-5      T 88   ☑ 88  

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 494us/step - loss: 0.1417 - acc: 0.9493 - val_loss: 0.1592 - val_acc: 0.9433
Q 772-7+215   T 980  ☑ 980 
Q 336-5+5     T 336  ☑ 336 
Q 962+4-66    T 900  ☒ 990 
Q 39-27+208   T 220  ☒ 210 
Q 766-81+638  T 1323 ☑ 1323
Q 889-8+5     T 886  ☑ 886 
Q 746+94-2    T 838  ☑ 838 
Q 8+225-74    T 159  ☑ 159 
Q 448+502-0   T 950  ☑ 950 
Q 7+26-9      T 24   ☑ 24  

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 495us/step - 

45000/45000 [==============================] - 22s 487us/step - loss: 0.0862 - acc: 0.9706 - val_loss: 0.1284 - val_acc: 0.9561
Q 812-78+8    T 742  ☑ 742 
Q 365-3+8     T 370  ☑ 370 
Q 53+216-0    T 269  ☑ 269 
Q 71-3+85     T 153  ☑ 153 
Q 28+446-10   T 464  ☑ 464 
Q 601-0+46    T 647  ☑ 647 
Q 902-80+502  T 1324 ☑ 1324
Q 64+944-682  T 326  ☑ 326 
Q 521+86-6    T 601  ☑ 601 
Q 12-7+799    T 804  ☒ 704 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 485us/step - loss: 0.0821 - acc: 0.9717 - val_loss: 0.1526 - val_acc: 0.9471
Q 474-64+5    T 415  ☑ 415 
Q 50+31-7     T 74   ☑ 74  
Q 46+639-54   T 631  ☑ 631 
Q 3+52-2      T 53   ☑ 53  
Q 332+6-7     T 331  ☑ 331 
Q 8+585-140   T 453  ☒ 443 
Q 269-83+99   T 285  ☑ 285 
Q 742-1+52    T 793  ☑ 793 
Q 264-119+7   T 152  ☑ 152 
Q 70-6+52     T 116  ☑ 116 

--------------------------------------------------
Iteratio

45000/45000 [==============================] - 22s 492us/step - loss: 0.0859 - acc: 0.9710 - val_loss: 0.1250 - val_acc: 0.9576
Q 645-6+33    T 672  ☑ 672 
Q 905-1+9     T 913  ☑ 913 
Q 385+846-39  T 1192 ☑ 1192
Q 73+7-66     T 14   ☑ 14  
Q 70+4-17     T 57   ☑ 57  
Q 16-2+16     T 30   ☑ 30  
Q 576-452+932 T 1056 ☑ 1056
Q 47+5-2      T 50   ☑ 50  
Q 226-17+63   T 272  ☑ 272 
Q 580-91+3    T 492  ☑ 492 

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 491us/step - loss: 0.0582 - acc: 0.9804 - val_loss: 0.1104 - val_acc: 0.9623
Q 76+85-23    T 138  ☑ 138 
Q 424+961-5   T 1380 ☑ 1380
Q 306+38-23   T 321  ☑ 321 
Q 89-18+967   T 1038 ☒ 1039
Q 6+915-149   T 772  ☑ 772 
Q 882-5+771   T 1648 ☑ 1648
Q 388-38+95   T 445  ☑ 445 
Q 94+14-4     T 104  ☑ 104 
Q 119-5+75    T 189  ☑ 189 
Q 137+822-604 T 355  ☑ 355 

--------------------------------------------------
Iteratio

...

45000/45000 [==============================] - 22s 496us/step - loss: 0.0279 - acc: 0.9909 - val_loss: 0.1063 - val_acc: 0.9700
Q 173+699-7   T 865  ☑ 865 
Q 907-66+494  T 1335 ☑ 1335
Q 36+0-6      T 30   ☑ 30  
Q 89-60+2     T 31   ☑ 31  
Q 73-56+704   T 721  ☑ 721 
Q 679+356-65  T 970  ☑ 970 
Q 7-0+314     T 321  ☑ 321 
Q 823-57+4    T 770  ☑ 770 
Q 196-0+239   T 435  ☑ 435 
Q 79-6+6      T 79   ☑ 79  

--------------------------------------------------
Iteration 146
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 494us/step - loss: 0.0246 - acc: 0.9926 - val_loss: 0.1156 - val_acc: 0.9692
Q 677-86+93   T 684  ☑ 684 
Q 534-32+756  T 1258 ☑ 1258
Q 73+598-4    T 667  ☑ 667 
Q 94-4+32     T 122  ☑ 122 
Q 164-66+459  T 557  ☑ 557 
Q 132+17-97   T 52   ☒ 54  
Q 17+36-11    T 42   ☑ 42  
Q 8+941-14    T 935  ☑ 935 
Q 36+68-2     T 102  ☑ 102 
Q 90+5-2      T 93   ☑ 93  

--------------------------------------------------
Iterati

45000/45000 [==============================] - 22s 493us/step - loss: 0.0207 - acc: 0.9936 - val_loss: 0.0968 - val_acc: 0.9725
Q 9-5+31      T 35   ☑ 35  
Q 85-71+5     T 19   ☑ 19  
Q 17+371-3    T 385  ☑ 385 
Q 9+252-53    T 208  ☑ 208 
Q 99-96+1     T 4    ☑ 4   
Q 7+748-19    T 736  ☑ 736 
Q 67-4+78     T 141  ☑ 141 
Q 9+940-906   T 43   ☒ 33  
Q 9+2-1       T 10   ☑ 10  
Q 463+9-1     T 471  ☑ 471 

--------------------------------------------------
Iteration 159
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 492us/step - loss: 0.0283 - acc: 0.9912 - val_loss: 0.0832 - val_acc: 0.9768
Q 362-3+711   T 1070 ☑ 1070
Q 89+7-3      T 93   ☑ 93  
Q 369-1+7     T 375  ☑ 375 
Q 95+656-92   T 659  ☑ 659 
Q 86-0+71     T 157  ☑ 157 
Q 28+446-10   T 464  ☑ 464 
Q 7+485-7     T 485  ☑ 485 
Q 17+36-11    T 42   ☑ 42  
Q 227-64+619  T 782  ☑ 782 
Q 30+6-36     T 0    ☑ 0   

--------------------------------------------------
Iterati

45000/45000 [==============================] - 22s 494us/step - loss: 0.0195 - acc: 0.9940 - val_loss: 0.1339 - val_acc: 0.9627
Q 54+713-5    T 762  ☑ 762 
Q 851+99-3    T 947  ☑ 947 
Q 4+626-6     T 624  ☑ 624 
Q 619-6+7     T 620  ☑ 620 
Q 493-80+36   T 449  ☑ 449 
Q 48+3-9      T 42   ☑ 42  
Q 457+66-41   T 482  ☑ 482 
Q 765+39-341  T 463  ☑ 463 
Q 918-18+3    T 903  ☑ 903 
Q 13+9-9      T 13   ☑ 13  

--------------------------------------------------
Iteration 172
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 494us/step - loss: 0.0234 - acc: 0.9925 - val_loss: 0.0912 - val_acc: 0.9758
Q 17+98-2     T 113  ☑ 113 
Q 5+56-2      T 59   ☑ 59  
Q 7-6+46      T 47   ☑ 47  
Q 504+742-0   T 1246 ☑ 1246
Q 98-1+13     T 110  ☑ 110 
Q 169+396-0   T 565  ☒ 556 
Q 5+645-0     T 650  ☑ 650 
Q 81-3+566    T 644  ☑ 644 
Q 4+485-89    T 400  ☑ 400 
Q 93+0-93     T 0    ☒ 1   

--------------------------------------------------
Iterati

Q 44+967-35   T 976  ☑ 976 
Q 488-1+89    T 576  ☒ 575 
Q 45-1+614    T 658  ☑ 658 
Q 437-3+148   T 582  ☑ 582 
Q 969-848+893 T 1014 ☑ 1014
Q 279-156+64  T 187  ☑ 187 
Q 6-1+982     T 987  ☑ 987 
Q 31-1+55     T 85   ☑ 85  
Q 10+98-8     T 100  ☒ 10  
Q 838-473+115 T 480  ☑ 480 

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 491us/step - loss: 0.0257 - acc: 0.9919 - val_loss: 0.1035 - val_acc: 0.9726
Q 49+4-9      T 44   ☑ 44  
Q 34+872-68   T 838  ☑ 838 
Q 831+400-11  T 1220 ☑ 1220
Q 151-51+55   T 155  ☑ 155 
Q 675-395+16  T 296  ☒ 295 
Q 884+0-9     T 875  ☑ 875 
Q 630-4+3     T 629  ☑ 629 
Q 50-35+70    T 85   ☑ 85  
Q 22-1+464    T 485  ☑ 485 
Q 81-14+4     T 71   ☑ 71  

--------------------------------------------------
Iteration 186
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 22s 492us/step 

45000/45000 [==============================] - 22s 496us/step - loss: 0.0189 - acc: 0.9944 - val_loss: 0.1280 - val_acc: 0.9677
Q 695+7-5     T 697  ☑ 697 
Q 968-8+82    T 1042 ☑ 1042
Q 675-5+50    T 720  ☑ 720 
Q 26-21+833   T 838  ☑ 838 
Q 992-76+4    T 920  ☑ 920 
Q 84+684-259  T 509  ☒ 599 
Q 326-0+962   T 1288 ☑ 1288
Q 928-366+996 T 1558 ☑ 1558
Q 568-7+468   T 1029 ☑ 1029
Q 755-293+6   T 468  ☑ 468 

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 23s 501us/step - loss: 0.0257 - acc: 0.9921 - val_loss: 0.0863 - val_acc: 0.9775
Q 48+873-6    T 915  ☑ 915 
Q 944-20+110  T 1034 ☑ 1034
Q 910-8+52    T 954  ☑ 954 
Q 199-71+64   T 192  ☑ 192 
Q 283+26-14   T 295  ☑ 295 
Q 700+10-6    T 704  ☑ 704 
Q 7+377-80    T 304  ☑ 304 
Q 6+479-421   T 64   ☒ 65  
Q 563+29-45   T 547  ☑ 547 
Q 6+76-2      T 80   ☑ 80  


In [20]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 908+3-3     T 908  ☑ 908 
Q 15+744-61   T 698  ☑ 698 
Q 970+2-7     T 965  ☑ 965 
Q 60+369-78   T 351  ☑ 351 
Q 6+3-6       T 3    ☑ 3   
Q 351+4-95    T 260  ☒ 250 
Q 5+8-7       T 6    ☑ 6   
Q 942+6-21    T 927  ☑ 927 
Q 5+3-3       T 5    ☑ 5   
Q 72+1-5      T 68   ☑ 68  
Testing Accuracy :  0.918


### 1.2 實驗使用不同的epoch和batch size訓練
* with batch = 256, epoch = 200

In [24]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 256
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 512)               1077248   
_________________________________________________________________
dense_11 (Dense)             (None, 2048)              1050624   
_________________________________________________________________
reshape_6 (Reshape)          (None, 4, 512)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 4, 512)            2099200   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 13)             6669      
Total params: 4,233,741
Trainable params: 4,233,741
Non-trainable params: 0
_________________________________________________________________


In [25]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 11s 240us/step - loss: 1.8106 - acc: 0.3684 - val_loss: 1.7413 - val_acc: 0.3817
Q 13-4+669    T 678  ☒ 13  
Q 447-308+78  T 217  ☒ 133 
Q 30-1+14     T 43   ☒ 23  
Q 8+983-476   T 515  ☒ 10  
Q 993-329+278 T 942  ☒ 103 
Q 949-9+120   T 1060 ☒ 103 
Q 644+2-8     T 638  ☒ 13  
Q 23+1-7      T 17   ☒ 2   
Q 267-9+846   T 1104 ☒ 127 
Q 101+31-0    T 132  ☒ 10  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 181us/step - loss: 1.6921 - acc: 0.3904 - val_loss: 1.6192 - val_acc: 0.4137
Q 50-42+0     T 8    ☒ 52  
Q 13-7+12     T 18   ☒ 12  
Q 25-2+799    T 822  ☒ 122 
Q 851+2-25    T 828  ☒ 12  
Q 722-714+72  T 80   ☒ 122 
Q 20+664-94   T 590  ☒ 122 
Q 27-5+8      T 30   ☒ 22  
Q 437+2-5     T 434  ☒ 

45000/45000 [==============================] - 8s 180us/step - loss: 0.6039 - acc: 0.7703 - val_loss: 0.5948 - val_acc: 0.7704
Q 886-65+38   T 859  ☒ 851 
Q 20+590-36   T 574  ☑ 574 
Q 53+1-4      T 50   ☑ 50  
Q 155-78+93   T 170  ☒ 172 
Q 7-5+183     T 185  ☑ 185 
Q 37-24+310   T 323  ☒ 319 
Q 3+36-2      T 37   ☑ 37  
Q 912+19-90   T 841  ☒ 831 
Q 75-59+3     T 19   ☒ 24  
Q 170-49+103  T 224  ☒ 225 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 180us/step - loss: 0.5722 - acc: 0.7838 - val_loss: 0.5640 - val_acc: 0.7833
Q 511+933-7   T 1437 ☒ 1438
Q 49+180-72   T 157  ☒ 158 
Q 69-35+8     T 42   ☒ 43  
Q 284-8+1     T 277  ☑ 277 
Q 24+69-3     T 90   ☒ 80  
Q 44+2-39     T 7    ☒ 1   
Q 442-91+307  T 658  ☒ 668 
Q 16-8+5      T 13   ☑ 13  
Q 2-0+123     T 125  ☑ 125 
Q 86+427-7    T 506  ☑ 506 

--------------------------------------------------
Iteration 

45000/45000 [==============================] - 8s 183us/step - loss: 0.2744 - acc: 0.8990 - val_loss: 0.3508 - val_acc: 0.8701
Q 48+9-1      T 56   ☒ 57  
Q 645-2+725   T 1368 ☑ 1368
Q 828+4-760   T 72   ☒ 16  
Q 7-6+291     T 292  ☑ 292 
Q 676+2-373   T 305  ☒ 306 
Q 418-3+12    T 427  ☒ 437 
Q 86-0+71     T 157  ☑ 157 
Q 576-1+67    T 642  ☑ 642 
Q 4+957-5     T 956  ☑ 956 
Q 55-9+48     T 94   ☑ 94  

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 181us/step - loss: 0.2384 - acc: 0.9151 - val_loss: 0.2890 - val_acc: 0.8934
Q 534-6+73    T 601  ☑ 601 
Q 4+485-0     T 489  ☒ 480 
Q 124-7+6     T 123  ☑ 123 
Q 86-0+71     T 157  ☑ 157 
Q 183-25+46   T 204  ☑ 204 
Q 782+5-196   T 591  ☒ 581 
Q 3+36-2      T 37   ☑ 37  
Q 2-0+302     T 304  ☑ 304 
Q 328+56-165  T 219  ☒ 227 
Q 358-19+77   T 416  ☑ 416 

--------------------------------------------------
Iteration 

Q 78-0+664    T 742  ☑ 742 
Q 56+843-8    T 891  ☑ 891 
Q 9+923-11    T 921  ☑ 921 
Q 407+150-95  T 462  ☒ 461 
Q 0+287-7     T 280  ☑ 280 
Q 17+98-2     T 113  ☑ 113 
Q 79-19+108   T 168  ☑ 168 
Q 11-3+325    T 333  ☑ 333 
Q 831-4+841   T 1668 ☑ 1668
Q 88-1+7      T 94   ☑ 94  

--------------------------------------------------
Iteration 41
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 180us/step - loss: 0.0951 - acc: 0.9687 - val_loss: 0.1422 - val_acc: 0.9493
Q 73-56+704   T 721  ☑ 721 
Q 93-4+127    T 216  ☑ 216 
Q 513+1-6     T 508  ☑ 508 
Q 913+19-5    T 927  ☑ 927 
Q 133-48+2    T 87   ☒ 86  
Q 437+110-51  T 496  ☒ 506 
Q 543-98+17   T 462  ☑ 462 
Q 74-7+922    T 989  ☑ 989 
Q 887+1-345   T 543  ☑ 543 
Q 446-26+5    T 425  ☑ 425 

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 181us/step - lo

45000/45000 [==============================] - 8s 179us/step - loss: 0.0698 - acc: 0.9768 - val_loss: 0.1366 - val_acc: 0.9514
Q 482-55+7    T 434  ☑ 434 
Q 370+4-29    T 345  ☑ 345 
Q 778-92+45   T 731  ☑ 731 
Q 877+3-8     T 872  ☑ 872 
Q 369-3+4     T 370  ☑ 370 
Q 30+43-8     T 65   ☑ 65  
Q 32+17-8     T 41   ☑ 41  
Q 337-213+383 T 507  ☒ 407 
Q 278+1-1     T 278  ☑ 278 
Q 99-4+386    T 481  ☑ 481 

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 180us/step - loss: 0.0613 - acc: 0.9804 - val_loss: 0.1321 - val_acc: 0.9548
Q 7+377-80    T 304  ☑ 304 
Q 12-5+6      T 13   ☑ 13  
Q 76-52+950   T 974  ☑ 974 
Q 192+519-82  T 629  ☒ 628 
Q 22+94-1     T 115  ☑ 115 
Q 77-43+4     T 38   ☑ 38  
Q 9-3+151     T 157  ☑ 157 
Q 4+28-1      T 31   ☑ 31  
Q 432+441-2   T 871  ☑ 871 
Q 366-47+35   T 354  ☑ 354 

--------------------------------------------------
Iteration 

...

Q 72+260-71   T 261  ☑ 261 
Q 4+951-19    T 936  ☒ 937 
Q 956-6+915   T 1865 ☒ 1855
Q 23+257-140  T 140  ☒ 139 
Q 47+486-6    T 527  ☑ 527 
Q 4+219-41    T 182  ☑ 182 
Q 695+7-5     T 697  ☑ 697 
Q 62+3-4      T 61   ☑ 61  
Q 887+1-345   T 543  ☑ 543 
Q 27-2+357    T 382  ☑ 382 

--------------------------------------------------
Iteration 135
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 179us/step - loss: 0.0135 - acc: 0.9965 - val_loss: 0.1077 - val_acc: 0.9689
Q 737-3+6     T 740  ☑ 740 
Q 732-0+9     T 741  ☑ 741 
Q 875-106+95  T 864  ☒ 854 
Q 9-0+89      T 98   ☑ 98  
Q 80-9+375    T 446  ☑ 446 
Q 733-91+1    T 643  ☑ 643 
Q 437-8+345   T 774  ☑ 774 
Q 954-6+95    T 1043 ☑ 1043
Q 827-8+612   T 1431 ☑ 1431
Q 4+103-1     T 106  ☑ 106 

--------------------------------------------------
Iteration 136
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 180us/step - 

45000/45000 [==============================] - 8s 179us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0802 - val_acc: 0.9780
Q 64-31+19    T 52   ☑ 52  
Q 161+437-333 T 265  ☑ 265 
Q 5-2+788     T 791  ☑ 791 
Q 80-26+896   T 950  ☑ 950 
Q 69-6+35     T 98   ☑ 98  
Q 443+856-68  T 1231 ☑ 1231
Q 410-34+246  T 622  ☑ 622 
Q 865-360+728 T 1233 ☑ 1233
Q 39-35+6     T 10   ☑ 10  
Q 246+3-93    T 156  ☑ 156 

--------------------------------------------------
Iteration 149
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 179us/step - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0802 - val_acc: 0.9776
Q 215-9+23    T 229  ☑ 229 
Q 392-9+4     T 387  ☑ 387 
Q 598-5+0     T 593  ☑ 593 
Q 85-4+59     T 140  ☑ 140 
Q 155-78+93   T 170  ☑ 170 
Q 22-7+77     T 92   ☑ 92  
Q 71+11-7     T 75   ☑ 75  
Q 826-77+596  T 1345 ☑ 1345
Q 8+259-176   T 91   ☒ 88  
Q 836+886-1   T 1721 ☑ 1721

--------------------------------------------------
Iteration

45000/45000 [==============================] - 8s 179us/step - loss: 0.0054 - acc: 0.9996 - val_loss: 0.0832 - val_acc: 0.9745
Q 41+616-263  T 394  ☑ 394 
Q 5+5-8       T 2    ☑ 2   
Q 966-796+50  T 220  ☑ 220 
Q 441+27-6    T 462  ☑ 462 
Q 366+37-3    T 400  ☑ 400 
Q 314+1-8     T 307  ☑ 307 
Q 93-4+127    T 216  ☑ 216 
Q 35-7+45     T 73   ☑ 73  
Q 9+544-32    T 521  ☑ 521 
Q 54+63-7     T 110  ☑ 110 

--------------------------------------------------
Iteration 162
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 179us/step - loss: 0.0041 - acc: 0.9997 - val_loss: 0.0848 - val_acc: 0.9742
Q 78+319-0    T 397  ☑ 397 
Q 56+439-71   T 424  ☑ 424 
Q 458-4+6     T 460  ☑ 460 
Q 97+12-2     T 107  ☑ 107 
Q 16-8+5      T 13   ☑ 13  
Q 40-7+60     T 93   ☑ 93  
Q 892-0+9     T 901  ☑ 901 
Q 24-5+6      T 25   ☑ 25  
Q 943+821-8   T 1756 ☑ 1756
Q 68-0+80     T 148  ☑ 148 

--------------------------------------------------
Iteration

45000/45000 [==============================] - 8s 184us/step - loss: 0.0083 - acc: 0.9983 - val_loss: 0.0837 - val_acc: 0.9757
Q 21+2-4      T 19   ☑ 19  
Q 246+51-6    T 291  ☑ 291 
Q 937+8-4     T 941  ☑ 941 
Q 580-0+436   T 1016 ☑ 1016
Q 724+952-455 T 1221 ☑ 1221
Q 3+50-23     T 30   ☑ 30  
Q 824-521+1   T 304  ☑ 304 
Q 827-3+247   T 1071 ☑ 1071
Q 58-0+411    T 469  ☑ 469 
Q 72-55+369   T 386  ☑ 386 

--------------------------------------------------
Iteration 175
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 182us/step - loss: 0.0043 - acc: 0.9993 - val_loss: 0.0874 - val_acc: 0.9749
Q 502+8-6     T 504  ☑ 504 
Q 456-95+2    T 363  ☑ 363 
Q 453-54+171  T 570  ☑ 570 
Q 684-60+2    T 626  ☑ 626 
Q 8+390-9     T 389  ☑ 389 
Q 6+769-514   T 261  ☑ 261 
Q 6+397-7     T 396  ☑ 396 
Q 813+75-6    T 882  ☑ 882 
Q 986+2-13    T 975  ☑ 975 
Q 66+2-0      T 68   ☑ 68  

--------------------------------------------------
Iteration

Q 818+2-740   T 80   ☒ 77  
Q 56+904-606  T 354  ☑ 354 
Q 12-10+5     T 7    ☑ 7   
Q 57+37-28    T 66   ☑ 66  
Q 857-96+384  T 1145 ☑ 1145
Q 212-57+46   T 201  ☑ 201 
Q 72+41-0     T 113  ☑ 113 
Q 298+34-27   T 305  ☑ 305 
Q 761+60-341  T 480  ☑ 480 
Q 78+43-0     T 121  ☑ 121 

--------------------------------------------------
Iteration 188
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 182us/step - loss: 8.7201e-04 - acc: 1.0000 - val_loss: 0.0764 - val_acc: 0.9785
Q 852+847-4   T 1695 ☑ 1695
Q 910-21+494  T 1383 ☑ 1383
Q 697-380+27  T 344  ☑ 344 
Q 24-9+6      T 21   ☑ 21  
Q 723+44-4    T 763  ☑ 763 
Q 365-30+88   T 423  ☑ 423 
Q 259-3+35    T 291  ☑ 291 
Q 531-2+570   T 1099 ☑ 1099
Q 343-45+0    T 298  ☑ 298 
Q 49+2-34     T 17   ☑ 17  

--------------------------------------------------
Iteration 189
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 8s 184us/ste

In [26]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 908+3-3     T 908  ☑ 908 
Q 15+744-61   T 698  ☑ 698 
Q 970+2-7     T 965  ☑ 965 
Q 60+369-78   T 351  ☑ 351 
Q 6+3-6       T 3    ☑ 3   
Q 351+4-95    T 260  ☑ 260 
Q 5+8-7       T 6    ☑ 6   
Q 942+6-21    T 927  ☑ 927 
Q 5+3-3       T 5    ☑ 5   
Q 72+1-5      T 68   ☑ 68  
Testing Accuracy :  0.919


### 1.3 實驗使用不同的epoch和batch size訓練
* with batch = 128, epoch = 100

In [28]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 512)               1077248   
_________________________________________________________________
dense_15 (Dense)             (None, 2048)              1050624   
_________________________________________________________________
reshape_8 (Reshape)          (None, 4, 512)            0         
_________________________________________________________________
lstm_16 (LSTM)               (None, 4, 512)            2099200   
_________________________________________________________________
time_distributed_8 (TimeDist (None, 4, 13)             6669      
Total params: 4,233,741
Trainable params: 4,233,741
Non-trainable params: 0
_________________________________________________________________


In [29]:
Train_Step(model, epoch = 100)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 349us/step - loss: 1.7653 - acc: 0.3753 - val_loss: 1.6689 - val_acc: 0.3925
Q 904-401+5   T 508  ☒ 407 
Q 29+780-399  T 410  ☒ 107 
Q 980-0+34    T 1014 ☒ 107 
Q 99+381-66   T 414  ☒ 107 
Q 2-0+123     T 125  ☒ 20  
Q 694-4+265   T 955  ☒ 137 
Q 700+10-6    T 704  ☒ 137 
Q 82+24-56    T 50   ☒ 207 
Q 773-733+962 T 1002 ☒ 177 
Q 381+144-87  T 438  ☒ 117 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 278us/step - loss: 1.4776 - acc: 0.4516 - val_loss: 1.2717 - val_acc: 0.5108
Q 2+817-76    T 743  ☒ 722 
Q 306-28+724  T 1002 ☒ 902 
Q 671-57+196  T 810  ☒ 702 
Q 13-0+611    T 624  ☒ 622 
Q 356+832-34  T 1154 ☒ 1022
Q 380-1+296   T 675  ☒ 692 
Q 985-906+5   T 84   ☒ 123 
Q 481+8-81    T 408  ☒

45000/45000 [==============================] - 12s 274us/step - loss: 0.3955 - acc: 0.8518 - val_loss: 0.4457 - val_acc: 0.8251
Q 846+4-79    T 771  ☒ 770 
Q 479-72+5    T 412  ☑ 412 
Q 746-33+268  T 981  ☒ 985 
Q 737-6+83    T 814  ☑ 814 
Q 30-1+14     T 43   ☑ 43  
Q 3+52-2      T 53   ☑ 53  
Q 87-55+18    T 50   ☒ 49  
Q 865-4+5     T 866  ☑ 866 
Q 61+133-57   T 137  ☒ 136 
Q 908-93+8    T 823  ☑ 823 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 0.3720 - acc: 0.8622 - val_loss: 0.3715 - val_acc: 0.8636
Q 434+9-8     T 435  ☑ 435 
Q 77-9+631    T 699  ☒ 609 
Q 112-75+10   T 47   ☒ 53  
Q 58+3-29     T 32   ☑ 32  
Q 24-1+689    T 712  ☑ 712 
Q 870+274-3   T 1141 ☒ 1142
Q 798-4+53    T 847  ☑ 847 
Q 925-838+8   T 95   ☒ 10  
Q 72-70+88    T 90   ☑ 90  
Q 599-4+445   T 1040 ☒ 1035

--------------------------------------------------
Iteratio

45000/45000 [==============================] - 12s 275us/step - loss: 0.1495 - acc: 0.9470 - val_loss: 0.2553 - val_acc: 0.9035
Q 298+34-27   T 305  ☑ 305 
Q 399-83+6    T 322  ☑ 322 
Q 14+620-598  T 36   ☒ 6   
Q 706+2-0     T 708  ☑ 708 
Q 51+858-7    T 902  ☑ 902 
Q 695-42+7    T 660  ☑ 660 
Q 81+654-29   T 706  ☑ 706 
Q 557+1-0     T 558  ☑ 558 
Q 17+149-91   T 75   ☒ 83  
Q 33-3+63     T 93   ☑ 93  

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 274us/step - loss: 0.1569 - acc: 0.9428 - val_loss: 0.2094 - val_acc: 0.9194
Q 2+851-99    T 754  ☒ 744 
Q 571+20-24   T 567  ☒ 568 
Q 127+48-80   T 95   ☒ 905 
Q 827-347+79  T 559  ☑ 559 
Q 8-4+13      T 17   ☑ 17  
Q 0+536-39    T 497  ☑ 497 
Q 499+888-118 T 1269 ☒ 1267
Q 5+5-8       T 2    ☑ 2   
Q 907-9+870   T 1768 ☑ 1768
Q 14+864-92   T 786  ☑ 786 

--------------------------------------------------
Iteratio

Q 1+610-73    T 538  ☒ 548 
Q 301+91-237  T 155  ☑ 155 
Q 693+0-28    T 665  ☒ 654 
Q 180+18-25   T 173  ☑ 173 
Q 994+41-162  T 873  ☑ 873 
Q 22+1-8      T 15   ☑ 15  
Q 176+8-35    T 149  ☒ 159 
Q 14+42-0     T 56   ☑ 56  
Q 69+153-5    T 217  ☑ 217 
Q 71-3+85     T 153  ☑ 153 

--------------------------------------------------
Iteration 41
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 272us/step - loss: 0.0935 - acc: 0.9670 - val_loss: 0.1469 - val_acc: 0.9485
Q 66+25-84    T 7    ☑ 7   
Q 14-5+7      T 16   ☑ 16  
Q 790-1+440   T 1229 ☒ 1239
Q 352-4+9     T 357  ☑ 357 
Q 4+88-18     T 74   ☑ 74  
Q 49-36+6     T 19   ☑ 19  
Q 541-1+692   T 1232 ☑ 1232
Q 34+257-85   T 206  ☑ 206 
Q 556-4+57    T 609  ☑ 609 
Q 3+7-3       T 7    ☑ 7   

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 274us/step - 

45000/45000 [==============================] - 12s 273us/step - loss: 0.0648 - acc: 0.9784 - val_loss: 0.2040 - val_acc: 0.9303
Q 537-299+182 T 420  ☒ 439 
Q 841-9+7     T 839  ☑ 839 
Q 84-54+0     T 30   ☑ 30  
Q 154-5+259   T 408  ☑ 408 
Q 0+74-74     T 0    ☑ 0   
Q 8+264-49    T 223  ☑ 223 
Q 3+960-3     T 960  ☑ 960 
Q 2+320-9     T 313  ☑ 313 
Q 9+3-2       T 10   ☑ 10  
Q 266-0+7     T 273  ☑ 273 

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 274us/step - loss: 0.0722 - acc: 0.9750 - val_loss: 0.1629 - val_acc: 0.9432
Q 627+2-82    T 547  ☑ 547 
Q 376-5+26    T 397  ☑ 397 
Q 126-30+200  T 296  ☒ 297 
Q 807-4+60    T 863  ☑ 863 
Q 629-67+94   T 656  ☑ 656 
Q 5+554-16    T 543  ☑ 543 
Q 35+6-20     T 21   ☑ 21  
Q 6+71-4      T 73   ☑ 73  
Q 105+82-3    T 184  ☑ 184 
Q 40+104-19   T 125  ☒ 126 

--------------------------------------------------
Iteratio

45000/45000 [==============================] - 12s 271us/step - loss: 0.0492 - acc: 0.9837 - val_loss: 0.1275 - val_acc: 0.9575
Q 847+669-41  T 1475 ☑ 1475
Q 934+363-742 T 555  ☒ 545 
Q 480-87+21   T 414  ☑ 414 
Q 3+225-37    T 191  ☑ 191 
Q 898-4+18    T 912  ☑ 912 
Q 10+212-8    T 214  ☑ 214 
Q 791-652+1   T 140  ☒ 130 
Q 45-2+836    T 879  ☑ 879 
Q 38-1+46     T 83   ☑ 83  
Q 898+564-8   T 1454 ☑ 1454

--------------------------------------------------
Iteration 68
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 0.0766 - acc: 0.9740 - val_loss: 0.1722 - val_acc: 0.9416
Q 572-56+776  T 1292 ☑ 1292
Q 41+203-88   T 156  ☑ 156 
Q 55-8+527    T 574  ☑ 574 
Q 6+769-514   T 261  ☒ 262 
Q 63-34+44    T 73   ☑ 73  
Q 98-23+1     T 76   ☑ 76  
Q 185-0+3     T 188  ☑ 188 
Q 90-17+5     T 78   ☑ 78  
Q 584-5+572   T 1151 ☒ 1251
Q 66-15+68    T 119  ☒ 129 

--------------------------------------------------
Iteratio

45000/45000 [==============================] - 12s 271us/step - loss: 0.0353 - acc: 0.9889 - val_loss: 0.1020 - val_acc: 0.9663
Q 935-30+7    T 912  ☑ 912 
Q 426+8-194   T 240  ☑ 240 
Q 8+2-0       T 10   ☑ 10  
Q 113+5-118   T 0    ☒ 1   
Q 233+2-5     T 230  ☑ 230 
Q 891-8+114   T 997  ☑ 997 
Q 629-67+94   T 656  ☑ 656 
Q 1+25-8      T 18   ☒ 19  
Q 581+3-3     T 581  ☑ 581 
Q 9+923-11    T 921  ☑ 921 

--------------------------------------------------
Iteration 81
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 273us/step - loss: 0.0360 - acc: 0.9888 - val_loss: 0.1256 - val_acc: 0.9589
Q 313-7+16    T 322  ☑ 322 
Q 975-6+908   T 1877 ☑ 1877
Q 4+951-19    T 936  ☑ 936 
Q 428-5+35    T 458  ☑ 458 
Q 79+2-73     T 8    ☒ 9   
Q 887+581-2   T 1466 ☑ 1466
Q 264-119+7   T 152  ☑ 152 
Q 168+291-8   T 451  ☑ 451 
Q 8+107-2     T 113  ☑ 113 
Q 701-84+52   T 669  ☒ 679 

--------------------------------------------------
Iteratio

Q 819-356+0   T 463  ☒ 453 
Q 737-49+749  T 1437 ☑ 1437
Q 933-78+8    T 863  ☑ 863 
Q 5-2+875     T 878  ☑ 878 
Q 86-8+2      T 80   ☑ 80  
Q 895-4+41    T 932  ☑ 932 
Q 276-5+33    T 304  ☑ 304 
Q 272+5-5     T 272  ☑ 272 
Q 2+73-7      T 68   ☒ 67  
Q 74-7+922    T 989  ☑ 989 

--------------------------------------------------
Iteration 94
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 275us/step - loss: 0.0457 - acc: 0.9848 - val_loss: 0.1439 - val_acc: 0.9557
Q 86+837-9    T 914  ☑ 914 
Q 644+659-2   T 1301 ☑ 1301
Q 675-395+16  T 296  ☒ 286 
Q 2+49-7      T 44   ☑ 44  
Q 93-12+71    T 152  ☑ 152 
Q 502+0-19    T 483  ☑ 483 
Q 571-36+5    T 540  ☑ 540 
Q 402+310-58  T 654  ☒ 544 
Q 56-1+257    T 312  ☑ 312 
Q 353-41+9    T 321  ☑ 321 

--------------------------------------------------
Iteration 95
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 275us/step - 

In [30]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 908+3-3     T 908  ☑ 908 
Q 15+744-61   T 698  ☑ 698 
Q 970+2-7     T 965  ☑ 965 
Q 60+369-78   T 351  ☑ 351 
Q 6+3-6       T 3    ☑ 3   
Q 351+4-95    T 260  ☑ 260 
Q 5+8-7       T 6    ☑ 6   
Q 942+6-21    T 927  ☑ 927 
Q 5+3-3       T 5    ☑ 5   
Q 72+1-5      T 68   ☑ 68  
Testing Accuracy :  0.875


## 2. 實驗不同位數的數字
* The digits of input number = 4

In [40]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 4
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+- '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_Data(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 14, 13)
(45000, 5, 13)
Validation Data:
(5000, 14, 13)
(5000, 5, 13)


In [41]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_19 (LSTM)               (None, 512)               1077248   
_________________________________________________________________
dense_19 (Dense)             (None, 2560)              1313280   
_________________________________________________________________
reshape_10 (Reshape)         (None, 5, 512)            0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 5, 512)            2099200   
_________________________________________________________________
time_distributed_10 (TimeDis (None, 5, 13)             6669      
Total params: 4,496,397
Trainable params: 4,496,397
Non-trainable params: 0
_________________________________________________________________


In [42]:
Train_Step(model, epoch = 300)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 19s 416us/step - loss: 1.7654 - acc: 0.3741 - val_loss: 1.6914 - val_acc: 0.3970
Q 6473+36-784    T 5725  ☒ 6641 
Q 624+33-3       T 654   ☒ 668  
Q 651-4+5736     T 6383  ☒ 664  
Q 9291-94+8578   T 17775 ☒ 1021 
Q 5433+8058-6552 T 6939  ☒ 1441 
Q 75-3+402       T 474   ☒ 74   
Q 0+9818-33      T 9785  ☒ 108  
Q 797-9+207      T 995   ☒ 764  
Q 1125+84-9      T 1200  ☒ 111  
Q 252-10+0       T 242   ☒ 221  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 331us/step - loss: 1.5029 - acc: 0.4463 - val_loss: 1.3028 - val_acc: 0.5057
Q 83+24-1        T 106   ☒ 12   
Q 987+688-469    T 1206  ☒ 111  
Q 96-93+5        T 8     ☒ 3    
Q 62-3+1         T 60    ☒ 63   
Q 6547+7-4       T 6550  ☒ 6541 
Q 7

45000/45000 [==============================] - 15s 325us/step - loss: 0.6160 - acc: 0.7679 - val_loss: 0.6673 - val_acc: 0.7427
Q 7647-109+6     T 7544  ☒ 7526 
Q 661-3+0        T 658   ☑ 658  
Q 105+215-78     T 242   ☒ 254  
Q 184-1+1        T 184   ☑ 184  
Q 670-0+22       T 692   ☑ 692  
Q 89+5514-24     T 5579  ☒ 5587 
Q 326-79+302     T 549   ☒ 543  
Q 97-82+556      T 571   ☒ 579  
Q 6073-688+9218  T 14603 ☒ 14551
Q 65+2060-61     T 2064  ☒ 2070 

--------------------------------------------------
Iteration 14
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 325us/step - loss: 0.5904 - acc: 0.7775 - val_loss: 0.6447 - val_acc: 0.7512
Q 738-30+5       T 713   ☑ 713  
Q 905-1+4026     T 4930  ☒ 4922 
Q 2665+8-8       T 2665  ☑ 2665 
Q 577-35+303     T 845   ☒ 853  
Q 7548+5-81      T 7472  ☒ 7464 
Q 2768-211+2     T 2559  ☒ 2566 
Q 406+888-6      T 1288  ☒ 1298 
Q 49+8553-3      T 8599  ☒ 8508 
Q 326+86-14      T 398   ☒

45000/45000 [==============================] - 15s 325us/step - loss: 0.2731 - acc: 0.9010 - val_loss: 0.4550 - val_acc: 0.8302
Q 1+547-7        T 541   ☑ 541  
Q 8130-13+7786   T 15903 ☒ 15803
Q 573-0+7057     T 7630  ☑ 7630 
Q 370+9-2        T 377   ☑ 377  
Q 672-47+61      T 686   ☒ 676  
Q 87+8-26        T 69    ☑ 69   
Q 6243-86+3733   T 9890  ☑ 9890 
Q 9073+16-1      T 9088  ☑ 9088 
Q 7878-312+10    T 7576  ☒ 7586 
Q 5417-7+4       T 5414  ☑ 5414 

--------------------------------------------------
Iteration 27
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 325us/step - loss: 0.2570 - acc: 0.9088 - val_loss: 0.4632 - val_acc: 0.8308
Q 441+5-4        T 442   ☑ 442  
Q 61+57-1        T 117   ☑ 117  
Q 69-4+920       T 985   ☑ 985  
Q 3649-3+6130    T 9776  ☑ 9776 
Q 53+64-1        T 116   ☑ 116  
Q 9697+9-99      T 9607  ☒ 9606 
Q 2345+19-319    T 2045  ☒ 2032 
Q 3599+388-41    T 3946  ☒ 4046 
Q 88+429-30      T 487   ☒

45000/45000 [==============================] - 15s 327us/step - loss: 0.1135 - acc: 0.9645 - val_loss: 0.5849 - val_acc: 0.8183
Q 2458-173+9485  T 11770 ☒ 11777
Q 42-8+7         T 41    ☑ 41   
Q 9393-518+85    T 8960  ☒ 8949 
Q 33+3986-0      T 4019  ☒ 4029 
Q 32+3020-23     T 3029  ☒ 3021 
Q 48+329-96      T 281   ☒ 272  
Q 9961-75+52     T 9938  ☒ 9939 
Q 7940-9+806     T 8737  ☑ 8737 
Q 541+45-444     T 142   ☒ 130  
Q 580+215-405    T 390   ☒ 477  

--------------------------------------------------
Iteration 40
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 330us/step - loss: 0.1251 - acc: 0.9591 - val_loss: 0.5263 - val_acc: 0.8407
Q 81+126-87      T 120   ☒ 110  
Q 467-53+5       T 419   ☑ 419  
Q 0+567-39       T 528   ☑ 528  
Q 1754+7-425     T 1336  ☒ 1343 
Q 6893+241-30    T 7104  ☒ 7004 
Q 88+6868-3191   T 3765  ☒ 2765 
Q 721+761-131    T 1351  ☒ 1483 
Q 91+3-73        T 21    ☑ 21   
Q 5814-9+861     T 6666  ☑

45000/45000 [==============================] - 15s 325us/step - loss: 0.0839 - acc: 0.9729 - val_loss: 0.6700 - val_acc: 0.8276
Q 989-82+8       T 915   ☒ 925  
Q 83+30-9        T 104   ☒ 105  
Q 67+2382-376    T 2073  ☒ 2189 
Q 645+79-0       T 724   ☒ 734  
Q 5432-12+884    T 6304  ☑ 6304 
Q 665-461+996    T 1200  ☒ 1100 
Q 86-7+645       T 724   ☒ 734  
Q 7282-829+7621  T 14074 ☒ 14177
Q 218-76+5       T 147   ☑ 147  
Q 190+58-69      T 179   ☒ 178  

--------------------------------------------------
Iteration 53
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 325us/step - loss: 0.0708 - acc: 0.9781 - val_loss: 0.5987 - val_acc: 0.8442
Q 869-720+2      T 151   ☒ 169  
Q 6561-1+6       T 6566  ☑ 6566 
Q 5814-9+861     T 6666  ☒ 6765 
Q 2293+7-3       T 2297  ☑ 2297 
Q 561+87-150     T 498   ☒ 505  
Q 341-5+246      T 582   ☑ 582  
Q 8212-1+36      T 8247  ☑ 8247 
Q 3+917-1        T 919   ☒ 928  
Q 401+230-8      T 623   ☑

...

45000/45000 [==============================] - 15s 325us/step - loss: 0.0250 - acc: 0.9924 - val_loss: 0.9327 - val_acc: 0.8418
Q 2815-2210+6    T 611   ☒ 661  
Q 454-2+8        T 460   ☑ 460  
Q 659-35+1       T 625   ☑ 625  
Q 452+955-7      T 1400  ☑ 1400 
Q 691+0-9        T 682   ☑ 682  
Q 9096+4-2267    T 6833  ☒ 6744 
Q 4263+9307-25   T 13545 ☒ 13548
Q 2665+8-8       T 2665  ☑ 2665 
Q 8929-5+28      T 8952  ☑ 8952 
Q 443-7+4        T 440   ☑ 440  

--------------------------------------------------
Iteration 242
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 326us/step - loss: 0.0186 - acc: 0.9944 - val_loss: 0.9771 - val_acc: 0.8373
Q 289+8462-42    T 8709  ☒ 8718 
Q 158+63-6       T 215   ☑ 215  
Q 2381+35-1      T 2415  ☑ 2415 
Q 1962+886-2     T 2846  ☒ 2836 
Q 4661-12+7      T 4656  ☑ 4656 
Q 44+766-9       T 801   ☑ 801  
Q 55-4+9732      T 9783  ☑ 9783 
Q 117+3333-646   T 2804  ☒ 2806 
Q 36-34+21       T 23    

Q 23-4+35        T 54    ☑ 54   
Q 54-2+9051      T 9103  ☑ 9103 
Q 96-6+576       T 666   ☑ 666  
Q 613-41+9669    T 10241 ☒ 10242
Q 5456+6493-8    T 11941 ☒ 11052
Q 93+394-25      T 462   ☑ 462  
Q 5930+73-7      T 5996  ☒ 6996 
Q 2675+3-103     T 2575  ☒ 2565 
Q 64+588-8       T 644   ☒ 655  
Q 69-1+5         T 73    ☑ 73   

--------------------------------------------------
Iteration 254
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 326us/step - loss: 0.0203 - acc: 0.9936 - val_loss: 0.9425 - val_acc: 0.8405
Q 14+190-40      T 164   ☑ 164  
Q 8289-48+1778   T 10019 ☑ 10019
Q 23-4+5923      T 5942  ☑ 5942 
Q 7229-714+5     T 6520  ☒ 6500 
Q 18+8616-46     T 8588  ☒ 8586 
Q 389+1638-53    T 1974  ☑ 1974 
Q 3+927-384      T 546   ☒ 547  
Q 3+4932-317     T 4618  ☒ 4513 
Q 77-12+589      T 654   ☑ 654  
Q 747-80+20      T 687   ☑ 687  

--------------------------------------------------
Iteration 255
Train on 45000 sample

45000/45000 [==============================] - 15s 325us/step - loss: 0.0247 - acc: 0.9921 - val_loss: 0.9470 - val_acc: 0.8411
Q 39+7780-93     T 7726  ☒ 7725 
Q 5699-129+2696  T 8266  ☒ 8275 
Q 68+84-69       T 83    ☑ 83   
Q 5923-7+3304    T 9220  ☑ 9220 
Q 965-886+1      T 80    ☒ 78   
Q 3808-111+17    T 3714  ☒ 3735 
Q 9479-86+406    T 9799  ☒ 9790 
Q 8756-9+5       T 8752  ☑ 8752 
Q 2227-677+3     T 1553  ☑ 1553 
Q 62+393-8       T 447   ☑ 447  

--------------------------------------------------
Iteration 267
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 325us/step - loss: 0.0180 - acc: 0.9942 - val_loss: 0.9281 - val_acc: 0.8453
Q 4+6-4          T 6     ☑ 6    
Q 780+82-76      T 786   ☑ 786  
Q 3+667-448      T 222   ☒ 220  
Q 2379-19+4631   T 6991  ☒ 6992 
Q 438-9+116      T 545   ☑ 545  
Q 39+38-1        T 76    ☒ 77   
Q 893-729+1071   T 1235  ☒ 1244 
Q 612+5-259      T 358   ☒ 354  
Q 867-9+4        T 862   

Q 4474-47+100    T 4527  ☑ 4527 
Q 4+71-36        T 39    ☑ 39   
Q 67+9193-92     T 9168  ☑ 9168 
Q 332+0-79       T 253   ☒ 255  
Q 4840-161+2711  T 7390  ☒ 7300 
Q 7643+3-2231    T 5415  ☒ 5401 
Q 1984+6069-36   T 8017  ☒ 8027 
Q 726+48-1       T 773   ☑ 773  
Q 4330-2894+356  T 1792  ☒ 1791 
Q 9-6+93         T 96    ☑ 96   

--------------------------------------------------
Iteration 279
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 325us/step - loss: 0.0207 - acc: 0.9934 - val_loss: 0.9663 - val_acc: 0.8416
Q 89+323-25      T 387   ☒ 396  
Q 6047+6437-38   T 12446 ☒ 12442
Q 158-33+9       T 134   ☑ 134  
Q 869+7-548      T 328   ☒ 327  
Q 7239+3380-467  T 10152 ☒ 10240
Q 6576-24+5494   T 12046 ☒ 11146
Q 447+3116-39    T 3524  ☒ 3525 
Q 435+646-260    T 821   ☒ 822  
Q 6699-4325+4    T 2378  ☒ 2381 
Q 99+476-7       T 568   ☒ 578  

--------------------------------------------------
Iteration 280
Train on 45000 sample

45000/45000 [==============================] - 15s 325us/step - loss: 0.0210 - acc: 0.9934 - val_loss: 0.9461 - val_acc: 0.8423
Q 80+5136-74     T 5142  ☒ 5141 
Q 3910+0-7       T 3903  ☒ 3813 
Q 5580+5731-75   T 11236 ☒ 11321
Q 73+8230-80     T 8223  ☒ 8213 
Q 772+6-114      T 664   ☒ 652  
Q 957-9+477      T 1425  ☑ 1425 
Q 1+49-31        T 19    ☒ 10   
Q 3857+91-4      T 3944  ☑ 3944 
Q 9565-7+56      T 9614  ☑ 9614 
Q 480+832-89     T 1223  ☒ 1207 

--------------------------------------------------
Iteration 292
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 326us/step - loss: 0.0180 - acc: 0.9942 - val_loss: 0.9449 - val_acc: 0.8408
Q 2855-880+10    T 1985  ☑ 1985 
Q 6937-1+6       T 6942  ☑ 6942 
Q 244-34+21      T 231   ☑ 231  
Q 7882+40-447    T 7475  ☒ 7476 
Q 403-364+3213   T 3252  ☒ 3262 
Q 864-11+9627    T 10480 ☑ 10480
Q 972-0+663      T 1635  ☑ 1635 
Q 743+7-7        T 743   ☑ 743  
Q 0+1686-712     T 974   

In [53]:
## Generate Testing Data
questions_test, expected_test = Generate_Data(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [54]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 727+4-42       T 689   ☑ 689  
Q 834+17-0       T 851   ☑ 851  
Q 95+3-25        T 73    ☒ 83   
Q 438+49-34      T 453   ☑ 453  
Q 3153+2-575     T 2580  ☒ 2570 
Q 13+4-1         T 16    ☑ 16   
Q 50+797-56      T 791   ☑ 791  
Q 982+345-99     T 1228  ☒ 1235 
Q 2+2717-0       T 2719  ☒ 2720 
Q 4+62-2         T 64    ☑ 64   
Testing Accuracy :  0.542


## 3. 三個數字相加相減
* Add/subtract 3 number (3 digits) together

In [59]:
def Generate_OtherData(DIGITS, DATA_SIZE, MAXLEN):
    questions = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE / 2:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        if((a+b)>=c):
            key = tuple(sorted((a, b, c)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}+{}-{}'.format(a, b, c)
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a + b - c)
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)
            
    while len(questions) >= DATA_SIZE / 2 and len(questions) < DATA_SIZE:    ## the amount of generated data is TRAINING_SIZE
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b, c = f(), f(), f()

        if(a>b):   #####
            # Skip any addition questions we've already seen
            # Also skip any such that x+Y == Y+x (hence the sorting).
            key = tuple(sorted((a, b, c)))
            if key in seen:
                continue
            seen.add(key)
            # Pad the data with spaces such that it is always MAXLEN.
            q = '{}-{}+{}'.format(a, b, c)    #####
            query = q + ' ' * (MAXLEN - len(q))
            ans = str(a - b + c)   #####
            # Answers can be of maximum size DIGITS + 1.
            ans += ' ' * (DIGITS + 1 - len(ans))
            if REVERSE:
                # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
                # space used for padding.)
                query = query[::-1]
            questions.append(query)
            expected.append(ans)
    print('Total addition questions:', len(questions))
    
    return questions, expected

In [60]:
# Generate different data

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+- '
ctable = CharacterTable(chars)
print(ctable)

questions, expected = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = TRAINING_SIZE, MAXLEN = MAXLEN)
x, y = Vectorization(questions, expected)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Generating data...
Total addition questions: 50000
Vectorization...
Training Data:
(45000, 11, 13)
(45000, 4, 13)
Validation Data:
(5000, 11, 13)
(5000, 4, 13)


In [61]:
RNN = layers.LSTM
HIDDEN_SIZE = 512   #128
BATCH_SIZE = 128
LAYERS = 1

model = Build_Model(RNN = RNN, HIDDEN_SIZE = HIDDEN_SIZE, BATCH_SIZE = BATCH_SIZE, LAYERS = LAYERS, MAXLEN = MAXLEN, DIGITS = DIGITS, chars = chars)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 512)               1077248   
_________________________________________________________________
dense_21 (Dense)             (None, 2048)              1050624   
_________________________________________________________________
reshape_11 (Reshape)         (None, 4, 512)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 4, 512)            2099200   
_________________________________________________________________
time_distributed_11 (TimeDis (None, 4, 13)             6669      
Total params: 4,233,741
Trainable params: 4,233,741
Non-trainable params: 0
_________________________________________________________________


In [63]:
Train_Step(model, epoch = 200)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 364us/step - loss: 1.7642 - acc: 0.3775 - val_loss: 1.6438 - val_acc: 0.4035
Q 862-24+35   T 873  ☒ 122 
Q 95+89-43    T 141  ☒ 900 
Q 465+72-5    T 532  ☒ 522 
Q 76+71-46    T 101  ☒ 72  
Q 5+595-22    T 578  ☒ 52  
Q 287-82+99   T 304  ☒ 102 
Q 1+936-20    T 917  ☒ 22  
Q 539+0-45    T 494  ☒ 520 
Q 377+6-0     T 383  ☒ 120 
Q 608+17-48   T 577  ☒ 102 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 1.4586 - acc: 0.4594 - val_loss: 1.2337 - val_acc: 0.5324
Q 820-9+3     T 814  ☑ 814 
Q 62-59+24    T 27   ☒ 40  
Q 91+40-92    T 39   ☒ 1   
Q 3+77-65     T 15   ☒ 1   
Q 96-53+9     T 52   ☒ 60  
Q 4+6-9       T 1    ☑ 1   
Q 52-0+507    T 559  ☒ 554 
Q 776-73+340  T 1043 ☒

45000/45000 [==============================] - 12s 276us/step - loss: 0.3562 - acc: 0.8662 - val_loss: 0.4184 - val_acc: 0.8408
Q 564-245+6   T 325  ☒ 329 
Q 296-3+127   T 420  ☒ 411 
Q 46+66-4     T 108  ☑ 108 
Q 778+599-258 T 1119 ☒ 1124
Q 637-8+90    T 719  ☒ 729 
Q 708-1+14    T 721  ☒ 711 
Q 911+13-8    T 916  ☑ 916 
Q 442-1+732   T 1173 ☒ 1172
Q 150+24-0    T 174  ☒ 173 
Q 8-0+600     T 608  ☑ 608 

--------------------------------------------------
Iteration 15
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.3274 - acc: 0.8774 - val_loss: 0.3693 - val_acc: 0.8591
Q 288-3+21    T 306  ☑ 306 
Q 848+0-327   T 521  ☒ 529 
Q 77+27-92    T 12   ☒ 22  
Q 721+905-4   T 1622 ☒ 1621
Q 1+191-4     T 188  ☑ 188 
Q 10-9+3      T 4    ☑ 4   
Q 83-72+601   T 612  ☑ 612 
Q 2+652-8     T 646  ☑ 646 
Q 9+966-3     T 972  ☑ 972 
Q 366-90+88   T 364  ☑ 364 

--------------------------------------------------
Iteratio

Q 1+488-8     T 481  ☑ 481 
Q 107-81+337  T 363  ☑ 363 
Q 1+95-43     T 53   ☒ 52  
Q 75-5+708    T 778  ☑ 778 
Q 37+59-51    T 45   ☑ 45  
Q 418+78-2    T 494  ☑ 494 
Q 34+355-76   T 313  ☑ 313 
Q 996+11-7    T 1000 ☑ 1000
Q 314+95-7    T 402  ☑ 402 
Q 68-57+17    T 28   ☒ 38  

--------------------------------------------------
Iteration 28
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.1249 - acc: 0.9555 - val_loss: 0.1474 - val_acc: 0.9450
Q 192-65+52   T 179  ☑ 179 
Q 337-67+71   T 341  ☑ 341 
Q 588+4-7     T 585  ☑ 585 
Q 60+128-4    T 184  ☑ 184 
Q 489+6-24    T 471  ☑ 471 
Q 578-289+135 T 424  ☑ 424 
Q 954-21+98   T 1031 ☒ 1021
Q 2+257-2     T 257  ☑ 257 
Q 290-2+6     T 294  ☑ 294 
Q 906+949-64  T 1791 ☒ 1790

--------------------------------------------------
Iteration 29
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - 

45000/45000 [==============================] - 12s 277us/step - loss: 0.0865 - acc: 0.9697 - val_loss: 0.1615 - val_acc: 0.9421
Q 74+41-72    T 43   ☑ 43  
Q 584-196+343 T 731  ☑ 731 
Q 800-1+2     T 801  ☑ 801 
Q 780-19+179  T 940  ☑ 940 
Q 49-14+76    T 111  ☑ 111 
Q 76+71-46    T 101  ☑ 101 
Q 712+2-8     T 706  ☑ 706 
Q 897+7-330   T 574  ☑ 574 
Q 577+660-704 T 533  ☒ 524 
Q 963-14+896  T 1845 ☑ 1845

--------------------------------------------------
Iteration 42
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.0831 - acc: 0.9712 - val_loss: 0.1807 - val_acc: 0.9362
Q 783+6-7     T 782  ☒ 781 
Q 943-67+4    T 880  ☑ 880 
Q 26-5+743    T 764  ☑ 764 
Q 67-42+279   T 304  ☑ 304 
Q 73+351-5    T 419  ☑ 419 
Q 76+16-3     T 89   ☑ 89  
Q 501+503-38  T 966  ☒ 856 
Q 150+1-4     T 147  ☑ 147 
Q 94-57+49    T 86   ☑ 86  
Q 561-0+47    T 608  ☑ 608 

--------------------------------------------------
Iteratio

Q 262-5+47    T 304  ☑ 304 
Q 67-36+65    T 96   ☑ 96  
Q 483+591-9   T 1065 ☒ 1055
Q 864-1+78    T 941  ☑ 941 
Q 41-6+654    T 689  ☑ 689 
Q 20+94-3     T 111  ☑ 111 
Q 19-2+53     T 70   ☑ 70  
Q 23-3+91     T 111  ☑ 111 
Q 666-504+2   T 164  ☒ 71  
Q 476+0-1     T 475  ☑ 475 

--------------------------------------------------
Iteration 55
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.0566 - acc: 0.9810 - val_loss: 0.1248 - val_acc: 0.9553
Q 75-5+708    T 778  ☑ 778 
Q 102-5+278   T 375  ☑ 375 
Q 917-19+8    T 906  ☑ 906 
Q 244-18+8    T 234  ☑ 234 
Q 90-30+370   T 430  ☑ 430 
Q 603-0+0     T 603  ☑ 603 
Q 701+313-335 T 679  ☑ 679 
Q 77+41-9     T 109  ☑ 109 
Q 506-6+2     T 502  ☑ 502 
Q 74-43+930   T 961  ☑ 961 

--------------------------------------------------
Iteration 56
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - 

...

Q 688-65+22   T 645  ☑ 645 
Q 562-1+960   T 1521 ☒ 1511
Q 707-18+3    T 692  ☑ 692 
Q 72+1-1      T 72   ☑ 72  
Q 168+40-62   T 146  ☑ 146 
Q 581-554+74  T 101  ☑ 101 
Q 32-6+860    T 886  ☑ 886 
Q 38-15+4     T 27   ☑ 27  
Q 95-34+72    T 133  ☑ 133 
Q 96-38+5     T 63   ☑ 63  

--------------------------------------------------
Iteration 136
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 270us/step - loss: 0.0217 - acc: 0.9936 - val_loss: 0.0978 - val_acc: 0.9698
Q 331-5+0     T 326  ☑ 326 
Q 750-71+1    T 680  ☑ 680 
Q 9+79-5      T 83   ☑ 83  
Q 50+261-27   T 284  ☒ 294 
Q 52+723-694  T 81   ☒ 71  
Q 687-61+430  T 1056 ☑ 1056
Q 36-17+28    T 47   ☑ 47  
Q 952+108-3   T 1057 ☑ 1057
Q 228-4+8     T 232  ☑ 232 
Q 196-87+7    T 116  ☑ 116 

--------------------------------------------------
Iteration 137
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step 

45000/45000 [==============================] - 12s 270us/step - loss: 0.0139 - acc: 0.9961 - val_loss: 0.0986 - val_acc: 0.9701
Q 57+362-65   T 354  ☑ 354 
Q 4+27-6      T 25   ☑ 25  
Q 850+4-0     T 854  ☑ 854 
Q 2+879-3     T 878  ☑ 878 
Q 659+2-6     T 655  ☑ 655 
Q 715+57-70   T 702  ☑ 702 
Q 62-4+2      T 60   ☑ 60  
Q 446-12+6    T 440  ☑ 440 
Q 12+331-1    T 342  ☑ 342 
Q 951-678+34  T 307  ☑ 307 

--------------------------------------------------
Iteration 150
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 271us/step - loss: 0.0256 - acc: 0.9920 - val_loss: 0.1863 - val_acc: 0.9499
Q 34+428-6    T 456  ☑ 456 
Q 216-33+562  T 745  ☑ 745 
Q 9-1+484     T 492  ☑ 492 
Q 148+65-0    T 213  ☑ 213 
Q 6+893-293   T 606  ☑ 606 
Q 484-81+446  T 849  ☑ 849 
Q 33+52-31    T 54   ☑ 54  
Q 5+350-45    T 310  ☑ 310 
Q 246+24-226  T 44   ☒ 45  
Q 414+75-3    T 486  ☑ 486 

--------------------------------------------------
Iterati

Q 574-7+171   T 738  ☑ 738 
Q 577+660-704 T 533  ☑ 533 
Q 781+494-4   T 1271 ☑ 1271
Q 8-2+723     T 729  ☑ 729 
Q 584-106+992 T 1470 ☑ 1470
Q 3+890-9     T 884  ☒ 883 
Q 593+268-183 T 678  ☒ 688 
Q 807-698+2   T 111  ☒ 121 
Q 820-9+3     T 814  ☑ 814 
Q 123-23+0    T 100  ☒ 90  

--------------------------------------------------
Iteration 163
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.0299 - acc: 0.9902 - val_loss: 0.1334 - val_acc: 0.9616
Q 864-1+78    T 941  ☑ 941 
Q 579-49+435  T 965  ☒ 966 
Q 4+559-8     T 555  ☑ 555 
Q 6-4+368     T 370  ☑ 370 
Q 45-15+430   T 460  ☑ 460 
Q 177+47-1    T 223  ☑ 223 
Q 3+81-66     T 18   ☑ 18  
Q 333-51+956  T 1238 ☑ 1238
Q 78-28+5     T 55   ☑ 55  
Q 53-5+244    T 292  ☑ 292 

--------------------------------------------------
Iteration 164
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 276us/step 

45000/45000 [==============================] - 12s 277us/step - loss: 0.0192 - acc: 0.9948 - val_loss: 0.1131 - val_acc: 0.9692
Q 576+8-16    T 568  ☑ 568 
Q 7+70-4      T 73   ☑ 73  
Q 81-1+550    T 630  ☑ 630 
Q 566+759-12  T 1313 ☑ 1313
Q 823-70+858  T 1611 ☑ 1611
Q 53+922-33   T 942  ☑ 942 
Q 466-16+96   T 546  ☑ 546 
Q 694-3+42    T 733  ☑ 733 
Q 0+949-4     T 945  ☑ 945 
Q 525+66-3    T 588  ☑ 588 

--------------------------------------------------
Iteration 177
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 12s 277us/step - loss: 0.0189 - acc: 0.9942 - val_loss: 0.1229 - val_acc: 0.9656
Q 97-7+402    T 492  ☑ 492 
Q 111+935-84  T 962  ☑ 962 
Q 22-7+43     T 58   ☑ 58  
Q 778+1-2     T 777  ☑ 777 
Q 6+69-75     T 0    ☑ 0   
Q 100+1-6     T 95   ☒ 96  
Q 531-5+6     T 532  ☑ 532 
Q 744+450-2   T 1192 ☒ 1292
Q 9+317-3     T 323  ☑ 323 
Q 74-8+4      T 70   ☑ 70  

--------------------------------------------------
Iterati

Q 86+543-51   T 578  ☑ 578 
Q 897-168+0   T 729  ☒ 728 
Q 40-38+0     T 2    ☒ 20  
Q 329+0-86    T 243  ☑ 243 
Q 762-4+586   T 1344 ☑ 1344
Q 633-55+264  T 842  ☑ 842 
Q 860-633+5   T 232  ☑ 232 
Q 883-0+667   T 1550 ☑ 1550
Q 72+120-157  T 35   ☒ 34  
Q 94-80+21    T 35   ☑ 35  

--------------------------------------------------
Iteration 190
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 281us/step - loss: 0.0173 - acc: 0.9945 - val_loss: 0.1035 - val_acc: 0.9705
Q 18+382-2    T 398  ☑ 398 
Q 18+9-5      T 22   ☑ 22  
Q 85-67+21    T 39   ☑ 39  
Q 781-78+6    T 709  ☑ 709 
Q 219-8+53    T 264  ☑ 264 
Q 14+60-4     T 70   ☑ 70  
Q 766+6-67    T 705  ☑ 705 
Q 40+2-32     T 10   ☑ 10  
Q 955+540-79  T 1416 ☑ 1416
Q 584-196+343 T 731  ☑ 731 

--------------------------------------------------
Iteration 191
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 281us/step 

In [64]:
## Generate Testing Data
questions_test, expected_test = Generate_OtherData(DIGITS = DIGITS, DATA_SIZE = 1000, MAXLEN = MAXLEN)
x_test, y_test = Vectorization(questions_test, expected_test)

Generating data...
Total addition questions: 1000
Vectorization...


In [65]:
## Testing
count_correct = 0
preds = model.predict_classes(x_test, verbose=0)
print("Visualize 10 Data (Total Testing Data = 1000)")
for i in range(1000):
    q = ctable.decode(x_test[i])
    correct = ctable.decode(y_test[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    if correct == guess:
        count_correct += 1
    if i < 10:
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)
    
print("Testing Accuracy : ",(float)(count_correct / len(x_test)))

Visualize 10 Data (Total Testing Data = 1000)
Q 749+643-4   T 1388 ☑ 1388
Q 55+6-44     T 17   ☑ 17  
Q 13+734-7    T 740  ☑ 740 
Q 0+787-26    T 761  ☑ 761 
Q 231+27-49   T 209  ☑ 209 
Q 4+12-3      T 13   ☑ 13  
Q 3+576-301   T 278  ☑ 278 
Q 322+8-5     T 325  ☑ 325 
Q 86+0-60     T 26   ☑ 26  
Q 22+70-20    T 72   ☑ 72  
Testing Accuracy :  0.886


# 總結
## 在Addition_Subtraction_rnn中採用lstm實現加減混合
## 實驗:
### 1. 採用四種不同的batch size和訓練epochs (兩個三位數相加相減)
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 97.82% / Testing acc (1000 testing data) = 0.906
###     * [ batch size = 64,   epoch = 200 ] : Validation acc = 97.77% / Testing acc (1000 testing data) = 0.918
###     * [ batch size = 256, epoch = 200 ] : Validation acc = 97.97% / Testing acc (1000 testing data) = 0.919
###     * [ batch size = 128, epoch = 100 ] : Validation acc = 96.53% / Testing acc (1000 testing data) = 0.875
### 2. 兩個四位數相加相減
###     * [ batch size = 128, epoch = 300 ] : Validation acc = 85.66% / Testing acc (1000 testing data) = 0.542
### 3. 三個三位數相加相減
###     * [ batch size = 128, epoch = 200 ] : Validation acc = 96.88% / Testing acc (1000 testing data) = 0.886 